Task: Sentiment analysis of amazon reviews

Columns description:

    label = good or bad - you have to predict that for test dataset
    title = review title
    content = review content

Most of the reviews are in English, but there are a few in other languages.

Metric: accuracy
baseline: 0.8721


In [44]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

SEED = 1337

Подготовка train

In [45]:
df_train = pd.read_csv('./train.csv')
df_train.head()

,Unnamed: 0,content,label,title
0,0,My Mother is the President of a Dallas book c...,good,OUTSTANDING!!!!!!!!
1,1,I have a 2 Year old boy and ever since he was...,bad,Pamper Baby Dry are more like Pamper baby wet!
2,2,I thought this movie was going to have an inc...,bad,"Wow, what a letdown"
3,3,THERE BEATS ARE DIFFERENT THAN NO LIMITS BUT ...,bad,CASH MONEY AND NO LIMIT ARE THA SAME THANG!
4,4,"Tess, a naive and young girl whose one night ...",good,how tess' motherhood experience change her lif...


In [46]:
df_train.label.value_counts(normalize=True)

bad     0.512083
good    0.487917
Name: label, dtype: float64

In [47]:
df_train.loc[df_train.label == 'bad', 'label'] = 0
df_train.loc[df_train.label == 'good', 'label'] = 1
df_train.head()

,Unnamed: 0,content,label,title
0,0,My Mother is the President of a Dallas book c...,1,OUTSTANDING!!!!!!!!
1,1,I have a 2 Year old boy and ever since he was...,0,Pamper Baby Dry are more like Pamper baby wet!
2,2,I thought this movie was going to have an inc...,0,"Wow, what a letdown"
3,3,THERE BEATS ARE DIFFERENT THAN NO LIMITS BUT ...,0,CASH MONEY AND NO LIMIT ARE THA SAME THANG!
4,4,"Tess, a naive and young girl whose one night ...",1,how tess' motherhood experience change her lif...


In [48]:
y_train = df_train.label.values
y_train = y_train.astype(int)

print('train', df_train.shape[0])

train 9600


In [49]:
df_train = df_train.astype('U')

Подготовка test

In [50]:
df_test = pd.read_csv('./test.csv')
df_test.loc[df_test.label == 'bad', 'label'] = 0
df_test.loc[df_test.label == 'good', 'label'] = 1
df_test.head()

,Unnamed: 0,content,label,title
0,0,I saw the video for Galang on BPM and was lau...,1,It'll get in your head ...
1,1,I hate this printer! It is nothing but proble...,0,Extremely unhappy
2,2,Theres nothing about this product I don't lik...,1,i got my moneys worth.
3,3,Story is disjointed and so at times difficult...,0,Anno dracula
4,4,"Christine Feehan does it again, for all of yo...",1,Great Reading


In [51]:
y_test = df_test.label.values
y_test = y_test.astype(int)
print('test', df_test.shape[0])

test 2401


In [52]:
df_test.label.value_counts(normalize=True)

0    0.506872
1    0.493128
Name: label, dtype: float64

In [53]:
pd.Series(y_train).value_counts()

0    4916
1    4684
dtype: int64

In [54]:
pd.Series(y_test).value_counts()

0    1217
1    1184
dtype: int64

In [55]:
df_test = df_test.astype('U')

Analysis

In [78]:
svc = LinearSVC(C=1,dual=False,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr',tol=0.00001,verbose=0,random_state=SEED)
parameters = {'penalty':['l1','l2']}

model1 = TfidfVectorizer(analyzer='char', use_idf=True, smooth_idf=True,
                                  lowercase=True, 
                                  stop_words=stopwords.words('english'),
                                  min_df=5,
                                  ngram_range=(1,4), norm='l2')
model2 = GridSearchCV(svc, parameters, cv=5)

model = Pipeline([('tfidf', model1), ('est', model2)])

In [82]:
model.fit(df_train.content, y_train)
print('train', metrics.accuracy_score(y_train, model.predict(df_train.content)))

train 0.968333333333


In [83]:
model.fit(df_test.content, y_test)
print('test', metrics.accuracy_score(y_test, model.predict(df_test.content)))

test 0.985839233653
